# Interfacing Veloxchem as SCF driver with Qiskit Nature

The aim of this tutorial is to show how to use VeloxChem as an SCF driver to generate the one and two electron integrals that will be converted to Qbits and can be used as an input for Qiskit Nature Quantum Algorithms and get molecular energies and properties

In [ ]:
import veloxchem as vlx
from qiskit_nature.operators import second_quantization
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.drivers.molecule import Molecule
import numpy as np

In [ ]:
geometry = mol_str = """
O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, 'sto-3g')

In [ ]:
nbas = vlx.MolecularBasis.get_dimensions_of_basis(basis, molecule)
nocc = molecule.number_of_electrons() // 2
V_nuc = molecule.nuclear_repulsion_energy()

print('Number of contracted basis functions:', nbas)
print('Number of doubly occupied molecular orbitals:', nocc)
print(f'Nuclear repulsion energy (in a.u.): {V_nuc : 14.12f}')

Getting the integrals 

In [ ]:
# overlap matrix
overlap_drv = vlx.OverlapIntegralsDriver()
S = overlap_drv.compute(molecule, basis).to_numpy()

# one-electron Hamiltonian
kinetic_drv = vlx.KineticEnergyIntegralsDriver()
T = kinetic_drv.compute(molecule, basis).to_numpy()

nucpot_drv = vlx.NuclearPotentialIntegralsDriver()
V = -nucpot_drv.compute(molecule, basis).to_numpy()

h = T + V 
h_alpha = h
h_beta = h_alpha

# two-electron Hamiltonian
eri_drv = vlx.ElectronRepulsionIntegralsDriver()
g = np.zeros((nbas, nbas, nbas, nbas))
g_aa = g
g_bb = g_aa
g_ab = g_aa
g_ba = g_ab
eri_drv.compute_in_mem(molecule, basis, g)




Classic SCF Driver solution for the electronic ground state energy

In [ ]:
# symmetric transformation
sigma, U = np.linalg.eigh(S)
X = np.einsum('ik,k,jk->ij', U, 1/np.sqrt(sigma), U)

In [ ]:
def get_MO_coeff(F):

    F_OAO = np.einsum('ki,kl,lj->ij', X, F, X)
    epsilon, C_OAO = np.linalg.eigh(F_OAO)    
    C = np.einsum("ik,kj->ij", X, C_OAO)
    
    return C



In [ ]:
max_iter = 50
conv_thresh = 1e-4

# initial guess from core Hamiltonian
C = get_MO_coeff(h)

print("iter      SCF energy    Error norm")

for iter in range(max_iter):
    
    D = np.einsum('ik,jk->ij', C[:, :nocc], C[:, :nocc])
        
    J = np.einsum('ijkl,kl->ij', g, D)
    K = np.einsum('ilkj,kl->ij', g, D)
    F = h + 2*J - K
    
    E = np.einsum('ij,ij->', h + F, D) + V_nuc

    # compute convergence metric
    F_MO = np.einsum('ki,kl,lj->ij', C, F, C)
    e_vec = np.reshape(F_MO[:nocc, nocc:], -1)
    error = np.linalg.norm(e_vec)

    print(f'{iter:>2d}  {E:16.8f}  {error:10.2e}')

    if error < conv_thresh:
        print('SCF iterations converged!')
        break
    
    C = get_MO_coeff(F)


Now we compare with the Qiskit built-in SCF Driver PySCFDriver 

In [ ]:
from qiskit_nature.settings import settings
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.units_type import UnitsType
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
settings.dict_aux_operators = True
driver = PySCFDriver(atom='O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116',
                     unit=UnitsType.ANGSTROM,
                     basis='sto3g')
problem = ElectronicStructureProblem(driver)

One and two electrons in Qiskit Nature format from PySCF

In [ ]:
properties = problem.driver.run()
electronic_energy = properties.get_property('ElectronicEnergy')
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis

one_body_ao = electronic_energy.get_electronic_integral(ElectronicBasis.AO, 1)
two_body_ao = electronic_energy.get_electronic_integral(ElectronicBasis.AO, 2)
print(one_body_ao)
print(two_body_ao)

We are aiming to get the fermionic operators looking like this so we can use a converter and a mapper to transform those into Qbits

In [ ]:
second_q_ops = problem.second_q_ops()['ElectronicEnergy']
print(second_q_ops)
problem.num_spin_orbitals
problem.num_particles


Importing all the modules needed

In [ ]:
from qiskit_nature.properties.second_quantization.electronic.integrals import (
    ElectronicIntegrals,
    OneBodyElectronicIntegrals,
    TwoBodyElectronicIntegrals,
    IntegralProperty,
)
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis
from qiskit_nature.properties.second_quantization.electronic.electronic_energy import ElectronicEnergy

Transformation of the one and two electron integrals coming from VeloxChem from atomic orbital (AO) basis to spin orbital (SO) basis and formatting to Qiskit Nature

In [ ]:
h1 = OneBodyElectronicIntegrals(ElectronicBasis.AO,(h_alpha,h_beta),ElectronicIntegrals.INTEGRAL_TRUNCATION_LEVEL == 1).to_spin()
h1_so = OneBodyElectronicIntegrals(ElectronicBasis.SO,h1,) ## Just a reformatting so the integrals has the qiskit required format
h2 = TwoBodyElectronicIntegrals(ElectronicBasis.AO,(g_aa,g_bb,g_ab,g_ba),ElectronicIntegrals.INTEGRAL_TRUNCATION_LEVEL == 1).to_spin()
h2_so = TwoBodyElectronicIntegrals(ElectronicBasis.SO,h2,)## Just a reformatting so the integrals has the qiskit required format
electronic_energy = ElectronicEnergy([h1_so,h2_so])
print(electronic_energy)

Electronic energy in the form of fermionic operators

In [ ]:
hamiltonian = electronic_energy.second_q_ops()['ElectronicEnergy']
print(hamiltonian)

In [ ]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

Electronic structure in the form of Qbits

In [ ]:
qbit_converter = QubitConverter(mapper=JordanWignerMapper())
q_bit_op = qbit_converter.convert(hamiltonian)
print(q_bit_op)

Solving the electronic structure with the VQE and using UCCSD as an ansatz

In [ ]:
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(14,(nocc,nocc),qbit_converter)
ansatz = UCCSD(qbit_converter,(nocc,nocc), 14,reps=2,initial_state=init_state)
#ansatz.decompose().draw('mpl', style='iqx') ## The ansatz representation is too large

Backend - Quantum computer simulator

In [ ]:
from qiskit import Aer # Backend for the simulator

backend = Aer.get_backend('aer_simulator_statevector')

Electronic Structure Algorithm solver

In [ ]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE

optimizer = SLSQP()

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(q_bit_op)
print(result.eigenvalue.real)

electronic_structure_result_uccsd = problem.interpret(result)
print(electronic_structure_result_uccsd)
## It takes extremely long in the Quantum Computer Simulator

Reducing by 2 the number of Qbits with the parity mapper

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper

parity_mapper = ParityMapper()
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)
## This converter will need as a second argument the number of particles, we can get that from the problem or the properties
qbit_op_parity = parity_converter.convert(hamiltonian,(nocc,nocc))
print(qbit_op_parity)
## This new qbit hamiltonian has 12 qbits instead of 14